# Curate .yml file types

This notebook automatically identifies entries in `nfdi4bioimage.yml` that are missing a `type` tag and checks if they contain presentation slides. It focuses on Zenodo entries, inspecting them for PowerPoint files or landscape-oriented PDFs. Entries identified as slides will be updated with `type: Slides`.

### Setup

First, we import the required libraries. We will use `ruamel.yaml` to load and save the YAML file while preserving its formatting and comments. `requests` is needed for API calls to Zenodo, and `pypdf` for inspecting PDF files.

In [1]:
try:
    from ruamel.yaml import YAML
except ImportError:
    !pip install ruamel.yaml
    from ruamel.yaml import YAML

try:
    import pypdf
except ImportError:
    !pip install pypdf
    import pypdf

import re
import requests
import io
from pypdf import PdfReader

### Load Data

We load the `nfdi4bioimage.yml` file from the `resources` directory.

In [2]:
yml_path = '../resources/nfdi4bioimage.yml'
yaml_loader = YAML()
yaml_loader.allow_duplicate_keys = True
yaml_loader.preserve_quotes = True
yaml_loader.indent(mapping=2, sequence=4, offset=2)

with open(yml_path, 'r', encoding='utf-8') as f:
    data = yaml_loader.load(f)

### Identify Candidate Entries

We find all entries that are missing a `type` (or have an empty `type`) and contain a link to Zenodo. These are the candidates we will inspect automatically.

In [3]:
entries_to_check = []
for entry in data['resources']:
    if 'type' not in entry or not entry['type']:
        urls = entry.get('url', [])
        if not isinstance(urls, list):
            urls = [urls]
        
        for url in urls:
            if re.search(r'zenodo\.org/(?:record|records)/', url):
                entries_to_check.append(entry)
                break

print(f"Found {len(entries_to_check)} Zenodo entries to check for slides.")

Found 194 Zenodo entries to check for slides.


### Define Helper Functions

These functions will help us interact with the Zenodo API to check for slide-related files.

In [4]:
def get_record_id(url):
    """Extracts the Zenodo record ID from a URL."""
    match = re.search(r'zenodo\.org/(?:record|records)/(\d+)', url)
    return match.group(1) if match else None

def contains_powerpoint(record_id):
    """Checks a Zenodo record for .ppt or .pptx files via its API."""
    api_url = f'https://zenodo.org/api/records/{record_id}'
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        record_data = response.json()
        
        for file_info in record_data.get('files', []):
            if file_info['key'].lower().endswith(('.pptx', '.ppt')):
                return True
    except requests.RequestException as e:
        print(f"Could not fetch Zenodo record {record_id}: {e}")
    return False

def contains_landscape_pdf(record_id):
    """Checks a Zenodo record for any landscape-oriented PDF files."""
    api_url = f'https://zenodo.org/api/records/{record_id}'
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        record_data = response.json()

        for file_info in record_data.get('files', []):
            if file_info['key'].lower().endswith('.pdf'):
                pdf_url = file_info['links']['self']
                pdf_response = requests.get(pdf_url)
                if pdf_response.ok:
                    with io.BytesIO(pdf_response.content) as pdf_file:
                        reader = PdfReader(pdf_file)
                        if len(reader.pages) > 0:
                            page = reader.pages[0]
                            if page.mediabox.width > page.mediabox.height:
                                return True
    except Exception as e:
        print(f"Could not process PDFs for record {record_id}: {e}")
    return False

### Process Entries and Update Type

Now we iterate through the entries. For each candidate, we check for PowerPoint files or landscape PDFs. If either is found, we set the `type` to `Slides`.

**Note:** This step involves network requests and may take some time to complete.

In [5]:
modified_entries_count = 0
for entry in data['resources']:
    if 'type' not in entry or not entry['type']:
        urls = entry.get('url', [])
        if not isinstance(urls, list):
            urls = [urls]

        for url in urls:
            record_id = get_record_id(url)
            if record_id:
                print(f"Checking '{entry['name']}' (Record: {record_id})...")
                if contains_powerpoint(record_id) or contains_landscape_pdf(record_id):
                    print(f"  -> Flagging '{entry['name']}' as Slides.")
                    entry['type'] = 'Slides'
                    modified_entries_count += 1
                    break

Checking 'NFDI4BIOIMAGE - National Research Data Infrastructure for Microscopy and Bioimage Analysis' (Record: 13168693)...


Checking '[GBI EOE VII] Five (or ten) must-have items for making IT infrastructure for managing bioimage data' (Record: 11318151)...


  -> Flagging '[GBI EOE VII] Five (or ten) must-have items for making IT infrastructure for managing bioimage data' as Slides.
Checking 'DALIA Interchange Format' (Record: 11521029)...


  -> Flagging 'DALIA Interchange Format' as Slides.
Checking 'Large Language Models: An Introduction for Life Scientists' (Record: 13379394)...


  -> Flagging 'Large Language Models: An Introduction for Life Scientists' as Slides.
Checking 'ChatGPT for Image Analysis' (Record: 13371196)...


...

Ignoring wrong pointing object 95 0 (offset 0)


Ignoring wrong pointing object 132 0 (offset 0)


  -> Flagging 'Internal ALM BioImage Analysis workshop 2023' as Slides.
Checking 'Open source AI Tools for bioimage analysis workshop (2024) @ICOB, Academia Sinica, Taiwan' (Record: 13284351)...


  -> Flagging 'Open source AI Tools for bioimage analysis workshop (2024) @ICOB, Academia Sinica, Taiwan' as Slides.
Checking 'FAQ Künstliche Intelligenz und gute wissenschaftliche Praxis' (Record: 14045172)...


Checking 'DNG in BioFormat opens in wrong resolution' (Record: 15933943)...


### Save Changes

Finally, if any entries were modified, we write the updated data structure back to the `nfdi4bioimage.yml` file. This will overwrite the original file.

In [6]:
if modified_entries_count > 0:
    with open(yml_path, 'w', encoding='utf-8') as f:
        yaml_loader.dump(data, f)
    print(f"\nSuccessfully updated {modified_entries_count} entries in {yml_path}.")
else:
    print("\nNo entries were updated.")


Successfully updated 79 entries in ../resources/nfdi4bioimage.yml.
